## TradingEnv-v0

### Open AI 'Gym' for reinforcement-learning based trading algorithms

This gym implements a very simple trading environment for reinforcement learning.

The gym provides daily observations based on real market data pulled from Quandl on, by default, the SPY etf.  An episode is defined as 252 contiguous days sampled from the overall dataset.  Each day is one 'step' within the gym and for each step, the algo has a choice:

 - SHORT (0)
 - FLAT (1)
 - LONG  (2)
 
If you trade, you will be charged, by default, 10 BPS of the size of your trade.  Thus, going from short to long costs twice as much as going from short to/from flat.  Not trading also has a default cost of 1 BPS per step.  Nobody said it would be easy!
 
At the beginning of your episode, you are allocated 1 unit of cash.  This is your starting Net Asset Value (NAV). 

### Beating the trading game 

For our purposes, we'll say that beating a buy & hold strategy, on average, over one hundred episodes will notch a win to the proud ai player.  We'll illustrate exactly what that means below.

### Let's look at some code using the environment


###  imports

In [ ]:
import gym
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import gym_trading


### create the environment

This may take a moment as we are pulling historical data from quandl.

In [ ]:
env = gym.make('trading-v0')
#env.time_cost_bps = 0 # 


### the trading model

Each time step is a day.  Each episode is 252 trading days - a year.  Each day, we can choose to be short (0), flat (1) or long (2) the single instrument in our trading universe.

Let's run through a day and stay flat.

In [ ]:
observation = env.reset()
done = False
navs = []
while not done:
    action = 1 # stay flat
    observation, reward, done, info = env.step(action)
    navs.append(info['nav'])
    if done:
        #print 'Annualized return: ',navs[len(navs)-1]-1
        pd.DataFrame(navs).plot()


### Note that you are charged just for playing - to the tune of 1 basis point per day!


### Rendering

For now, no rendering has been implemented for this gym, but with each step, the following datum are provided which you can easily graph and otherwise visualize as we see above with the NAV:

 - pnl - how much did we make or lose between yesterday and today?
 - costs  - how much did we pay in costs today
 - nav    - our current nav
 

## utility methods: running strategies once or repeatedly

Although the gym can be 'exercised' directly as seen above, we've also written utility methods which allow for the running of a strategy once or over many episodes, facilitating training or other sorts of analysis.

To utilize these methods, strategies should be exposed as a function or lambda with the following signature:

`Action a = strategy( observation, environment )`
    
Below, we define some simple strategies and look briefly at their behavior to better understand the trading gym. 

In [ ]:
import trading_env as te

stayflat     = lambda o,e: 1   # stand pat
buyandhold   = lambda o,e: 2   # buy on day #1 and hold
randomtrader = lambda o,e: e.action_space.sample() # retail trader

# to run singly, we call run_strat.  we are returned a dataframe containing 
#  all steps in the sim.
env = env.unwrapped
bhdf = env.run_strat(buyandhold)

print(bhdf.head())

# we can easily plot our nav in time:
bhdf.bod_nav.plot(title='buy & hold nav')


### running the same strategy multiple times will likely yield different results as underlying data changes

In [ ]:
env.run_strat(buyandhold).bod_nav.plot(title='same strat, different results')
env.run_strat(buyandhold).bod_nav.plot()
env.run_strat(buyandhold).bod_nav.plot()

### comparing the buyandhold and random traders

In [ ]:
# running a strategy multiple times should yield insights 
#   into its expected behavior or give it oppty to learn
bhdf = env.run_strats(buyandhold,100)
rdf = env.run_strats(randomtrader,100)

comparo = pd.DataFrame({'buyhold':bhdf.mean(),
                        'random': rdf.mean()})
comparo

## Object of the game

From the above examples, we can see that buying and holding will, over the long run, give you the market return with low costs.

Randomly trading will instead destroy value rather quickly as costs overwhelm.

### So, what does it mean to win the trading game?  

For our purposes, we'll say that beating a buy & hold strategy, on average, over one hundred episodes will notch a win to the proud ai player.

To support this, the trading environment maintains the *mkt_return* which can be compared with the *sim_return*.

Note that the *mkt_return* is frictionless while the *sim_return* incurs both trading costs and the decay cost of 1 basis point per day, so overcoming the hurdle we've set here should be challenging.


### Playing the game: purloined policy gradients

I've taken and adapted (see [code](policy_gradient.py) for details) a policy gradient implementation based on tensorflow to try to play the single-instrument trading game.  Let's see how it does.

In [1]:
import gym
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import gym_trading

env = gym.make('trading-v0')
#env.time_cost_bps = 0 # 

env = env.unwrapped



import tensorflow as tf
import policy_gradient

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: Environment '<class 'gym_trading.envs.trading_env.TradingEnv'>' has deprecated methods. Compatibility code invoked.


/Users/andrewplaate/miniconda3/envs/mlp/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
INFO:policy_gradient:policy_gradient logger started.


In [2]:
# create the tf session
sess = tf.InteractiveSession()

# create policygradient
pg = policy_gradient.PolicyGradient(sess, obs_dim=6, num_actions=2, learning_rate=1e-2 )

# and now let's train it and evaluate its progress.  NB: this could take some time...
df,sf = pg.train_model(env, episodes=25001, log_freq=100)#, load_model=True)


[[-0.49991614  0.5000838 ]]
[[-0.49941486  0.50058514]]
[[ 0.49842551  0.50157446]]
[[ 0.49901521  0.50098473]]
[[ 0.49953431  0.50046563]]
[[ 0.49965563  0.5003444 ]]
[[-0.50106388  0.49893612]]
[[ 0.49942335  0.50057667]]
[[-0.50019896  0.49980104]]
[[ 0.49930105  0.50069892]]
[[-0.49957088  0.50042915]]
[[ 0.49964121  0.50035876]]
[[-0.49949309  0.500507  ]]
[[ 0.49933785  0.50066215]]
[[-0.49946028  0.50053978]]
[[ 0.49873766  0.50126237]]
[[ 0.49873304  0.50126696]]
[[-0.49955633  0.5004437 ]]
[[ 0.49874917  0.50125086]]
[[-0.49853557  0.50146443]]
[[ 0.49834865  0.50165135]]
[[ 0.49815175  0.50184828]]
[[-0.49801823  0.50198179]]
[[ 0.49799475  0.50200522]]
[[ 0.49797034  0.50202966]]
[[ 0.49809912  0.50190085]]
[[ 0.49818701  0.50181293]]
[[ 0.49825561  0.50174439]]
[[ 0.49869716  0.50130284]]
[[ 0.49920103  0.50079894]]
[[ 0.49980772  0.50019228]]
[[-0.50023741  0.49976254]]
[[ 0.50185168  0.49814835]]
[[-0.49992704  0.5000729 ]]
[[ 0.49999711  0.50000292]]
[[ 0.50008601  0.499

[[ 0.48757425 -0.51242578]]
[[ 0.48850712 -0.51149285]]
[[ 0.48963928 -0.51036072]]
[[ 0.4899424  -0.51005763]]
[[ 0.48984289 -0.51015717]]
[[ 0.49003452 -0.50996548]]
[[ 0.48969057 -0.5103094 ]]
[[ 0.48937371 -0.51062632]]
[[ 0.48903584 -0.51096416]]
[[ 0.48843503 -0.51156497]]
[[ 0.48819488 -0.51180512]]
[[ 0.48862162 -0.51137841]]
[[ 0.48842436 -0.51157564]]
[[ 0.48850912 -0.51149088]]
[[ 0.48854756 -0.51145244]]
[[ 0.48884833 -0.51115173]]
[[ 0.48895174 -0.51104826]]
[[ 0.48862097 -0.511379  ]]
[[ 0.48813951 -0.51186055]]
[[ 0.48770636 -0.51229358]]
[[ 0.48713237 -0.51286763]]
[[ 0.48697597 -0.51302403]]
[[ 0.48670837 -0.51329172]]
[[ 0.48654398 -0.51345605]]
[[ 0.48732376 -0.51267618]]
[[ 0.48766196 -0.51233804]]
[[ 0.48690358 -0.51309639]]
[[ 0.48558542 -0.51441461]]
[[ 0.48542881 -0.51457125]]
[[ 0.48530093 -0.5146991 ]]
[[ 0.48500487 -0.5149951 ]]
[[ 0.48537892 -0.51462102]]
[[ 0.48526776 -0.51473218]]
[[ 0.48509651 -0.51490343]]
[[ 0.48510107 -0.51489896]]
[[ 0.48553619 -0.514

[[ 0.50052631 -0.49947366]]
[[ 0.50001168 -0.49998838]]
[[ 0.49932584 -0.50067413]]
[[ 0.49802792 -0.50197202]]
[[ 0.49834934 -0.50165063]]
[[ 0.4992727  -0.50072736]]
[[ 0.49981445 -0.50018549]]
[[ 0.49976569 -0.50023437]]
[[ 0.50022572 -0.49977428]]
[[ 0.50257272 -0.49742723]]
[[ 0.49962765 -0.50037241]]
[[ 0.49943519 -0.50056481]]
[[ 0.50037056 -0.49962953]]
[[ 0.50048721 -0.49951279]]
[[ 0.50232899 -0.49767101]]
[[ 0.49962392 -0.50037605]]
[[ 0.50156128 -0.49843872]]
[[ 0.49912032 -0.5008797 ]]
[[ 0.50104773 -0.49895233]]
[[ 0.50072479 -0.4992753 ]]
[[ 0.49979329 -0.50020671]]
[[ 0.49801588 -0.50198418]]
[[ 0.49748811 -0.50251186]]
[[ 0.49932456 -0.50067538]]
[[ 0.49918455 -0.50081539]]
[[ 0.50129479 -0.49870518]]
[[ 0.50061572 -0.49938431]]
[[ 0.49883547 -0.5011645 ]]
[[ 0.49998349 -0.50001657]]
[[ 0.50020826 -0.49979171]]
[[ 0.50138277 -0.49861723]]
[[ 0.50075758 -0.4992424 ]]
[[ 0.50193352 -0.49806648]]
[[ 0.5009613  -0.49903873]]
[[ 0.49823344 -0.50176656]]
[[ 0.49732357 -0.502

[[ 0.50051558 -0.49948445]]
[[ 0.50046486 -0.49953514]]
[[ 0.50046873 -0.49953127]]
[[ 0.50063396  0.49936596]]
[[ 0.50012863 -0.49987137]]
[[ 0.50075513 -0.49924484]]
[[ 0.5015955  -0.49840444]]
[[-0.50055212 -0.49944782]]
[[ 0.50002587 -0.49997419]]
[[ 0.50134158 -0.49865842]]
[[ 0.50037742 -0.49962258]]
[[ 0.49984127 -0.50015879]]
[[-0.49919686 -0.50080323]]
[[ 0.50091445 -0.49908555]]
[[ 0.50115383  0.4988462 ]]
[[ 0.50036949 -0.49963054]]
[[-0.50017148 -0.49982849]]
[[ 0.50020331 -0.49979663]]
[[ 0.49928632 -0.50071371]]
[[ 0.50069082 -0.49930915]]
[[ 0.50012988 -0.49987015]]
[[ 0.50030816 -0.49969184]]
[[ 0.49969143 -0.50030857]]
[[ 0.5013321 -0.4986679]]
[[-0.49966225 -0.50033772]]
[[ 0.50126928 -0.49873075]]
[[ 0.5005638  -0.49943623]]
[[ 0.50035936 -0.49964064]]
[[ 0.49966678 -0.50033325]]
[[ 0.500027   -0.49997303]]
[[ 0.50041991 -0.49958009]]
[[ 0.50013202 -0.49986801]]
[[ 0.50022966 -0.49977031]]
[[ 0.49990126 -0.50009871]]
[[-0.4997378 -0.5002622]]
[[ 0.50006539 -0.4999345

[[-0.49591139  0.50408858]]
[[-0.49906671  0.50093329]]
[[-0.49743697  0.502563  ]]
[[-0.49793771  0.50206226]]
[[-0.49618012  0.50381988]]
[[-0.49510235  0.50489765]]
[[-0.49763328  0.50236672]]
[[-0.49485096  0.50514907]]
[[-0.49442196  0.50557804]]
[[-0.49546617  0.50453383]]
[[-0.49556801  0.50443202]]
[[-0.49568504  0.5043149 ]]
[[-0.49287981  0.50712013]]
[[-0.49823987  0.50176013]]
[[-0.49901387  0.50098616]]
[[-0.50015128  0.49984869]]
[[-0.49596375  0.50403625]]
[[-0.50047064  0.49952933]]
[[-0.49781919  0.50218081]]
[[-0.49957877  0.50042117]]
[[-0.49994472  0.50005531]]
[[-0.49971706  0.500283  ]]
[[-0.50163758  0.49836248]]
[[-0.49837235  0.50162774]]
[[-0.499686  0.500314]]
[[-0.49558476  0.50441515]]
[[ 0.50152773 -0.49847227]]
[[-0.50115865  0.49884135]]
[[-0.49913388  0.50086612]]
[[-0.49741828  0.50258172]]
[[-0.5003987   0.49960124]]
[[-0.5001806   0.49981946]]
[[ 0.49825478  0.50174528]]
[[ 0.50022346 -0.49977654]]
[[ 0.49779654 -0.50220346]]
[[ 0.4982492  -0.5017507

...

[[ 0.48905298 -0.51094699]]
[[ 0.48955649 -0.51044351]]
[[ 0.4897362  -0.51026386]]
[[ 0.49044511 -0.50955492]]
[[ 0.49029863 -0.50970131]]
[[ 0.4902716  -0.50972843]]
[[ 0.49065182 -0.50934809]]
[[ 0.48931444 -0.51068556]]
[[ 0.48949471 -0.51050532]]
[[ 0.49065199 -0.50934803]]
[[ 0.49144107 -0.50855893]]
[[ 0.49117666 -0.50882334]]
[[ 0.49096397 -0.50903606]]
[[ 0.49214587 -0.50785416]]
[[ 0.49155533 -0.50844467]]
[[ 0.49241254 -0.50758743]]
[[ 0.49303752 -0.50696248]]
[[ 0.49235535 -0.50764459]]
[[ 0.49357522 -0.50642473]]
[[ 0.49125224 -0.50874782]]
[[ 0.49123046 -0.50876951]]
[[ 0.49154299 -0.50845695]]
[[ 0.49114305 -0.50885689]]
[[ 0.49130446 -0.5086956 ]]
[[ 0.49306425 -0.50693578]]
[[ 0.4921782 -0.5078218]]
[[ 0.49186435 -0.50813562]]
[[ 0.49511591 -0.50488406]]
[[ 0.49831513 -0.5016849 ]]
[[ 0.49387476 -0.50612521]]
[[ 0.49486822 -0.50513184]]
[[ 0.49608424 -0.50391567]]
[[ 0.49341819 -0.50658184]]
[[ 0.49267161 -0.50732839]]
[[ 0.49488139 -0.50511861]]
[[ 0.49842706 -0.50157

[[ 0.50017756  0.4998225 ]]
[[ 0.50032258 -0.49967736]]
[[-0.50051302  0.49948698]]
[[-0.50059175  0.4994083 ]]
[[-0.50034553  0.4996545 ]]
[[ 0.50188845 -0.49811152]]
[[ 0.49921831 -0.50078166]]
[[-0.49921507  0.50078499]]
[[-0.49962991  0.50037014]]
[[-0.49670237  0.50329757]]
[[-0.49822709  0.50177294]]
[[-0.49718681  0.50281316]]
[[-0.49651897  0.50348109]]
[[-0.49745911  0.50254095]]
[[-0.49626979  0.50373018]]
[[-0.49767771  0.50232232]]
[[-0.49619776  0.50380218]]
[[-0.49447772  0.50552231]]
[[-0.49313736  0.50686264]]
[[-0.49539778  0.50460225]]
[[-0.49781469  0.5021854 ]]
[[-0.49412364  0.5058763 ]]
[[-0.49425057  0.50574952]]
[[-0.49399573  0.50600421]]
[[-0.49387503  0.50612497]]
[[-0.49543774  0.50456226]]
[[-0.49359354  0.50640643]]
[[-0.49379194  0.50620806]]
[[-0.49556991  0.50443006]]
[[-0.49474874  0.50525123]]
[[-0.49454358  0.50545639]]
[[-0.49236062  0.50763935]]
[[-0.49290752  0.50709242]]
[[-0.4933807   0.50661933]]
[[-0.49412721  0.50587279]]
[[-0.49345219  0.506

[[-0.49756381  0.50243622]]
[[-0.49746203  0.50253803]]
[[-0.49737224  0.50262767]]
[[-0.49850991  0.50149006]]
[[-0.49735871  0.50264132]]
[[-0.49842978  0.50157022]]
[[-0.49922362  0.50077641]]
[[-0.49824947  0.50175053]]
[[-0.49910212  0.50089788]]
[[-0.49836183  0.50163823]]
[[-0.49874601  0.50125402]]
[[-0.49769861  0.50230139]]
[[-0.49732402  0.50267601]]
[[-0.49801269  0.50198728]]
[[-0.4970952   0.50290489]]
[[-0.49766764  0.50233233]]
[[-0.49871075  0.50128931]]
[[-0.49867442  0.50132561]]
[[-0.49936455  0.5006355 ]]
[[-0.49750748  0.50249249]]
[[-0.49797451  0.50202554]]
[[-0.49872881  0.50127119]]
[[-0.4982762   0.50172383]]
[[-0.49754849  0.50245148]]
[[-0.49711931  0.50288075]]
[[-0.49783644  0.50216347]]
[[-0.49747619  0.50252378]]
[[-0.49576962  0.50423032]]
[[-0.49645701  0.50354296]]
[[-0.49673748  0.50326258]]
[[-0.49590877  0.5040912 ]]
[[-0.49583226  0.50416774]]
[[-0.49762449  0.50237542]]
[[-0.4963066  0.5036934]]
[[-0.49730459  0.50269532]]
[[-0.49748626  0.50251

[[-0.49238005  0.50761992]]
[[-0.49287918  0.50712091]]
[[-0.49215561  0.50784439]]
[[-0.48997572  0.51002425]]
[[-0.48822746  0.51177251]]
[[-0.49108243  0.50891763]]
[[-0.49013674  0.5098632 ]]
[[-0.49042588  0.50957412]]
[[-0.49017403  0.509826  ]]
[[-0.49369228  0.50630772]]
[[-0.48920527  0.51079476]]
[[-0.49247408  0.50752586]]
[[-0.4929052   0.50709474]]
[[-0.49163222  0.50836784]]
[[ 0.50662947 -0.49337044]]
[[-0.49568313  0.50431681]]
[[-0.49409953  0.50590056]]
[[-0.49411717  0.5058828 ]]
[[-0.49315628  0.50684369]]
[[-0.49270278  0.50729722]]
[[-0.49247566  0.50752443]]
[[-0.49107823  0.5089218 ]]
[[-0.49190792  0.50809205]]
[[ 0.51013899 -0.48986104]]
[[-0.49694765  0.50305241]]
[[-0.49528763  0.50471234]]
[[-0.49445394  0.50554609]]
[[ 0.49473658  0.50526339]]
[[-0.49562809  0.50437188]]
[[-0.49403736  0.50596261]]
[[-0.49400786  0.50599223]]
[[-0.49587259  0.50412738]]
[[-0.49513626  0.50486374]]
[[-0.49478489  0.50521505]]
[[-0.49473634  0.50526363]]
[[-0.49358734  0.506

KeyboardInterrupt: 

### Results

Policy gradients beat the trading game!  That said, it doesn't work every time and it seems, looking at the charts below, as though it's a bit of a lucky thing.  But luck counts in the trading game as in life!


In [ ]:
sf['net'] = sf.simror - sf.mktror
#sf.net.plot()
sf.net.expanding().mean().plot()
sf.net.rolling(100).mean().plot()

In [ ]:
sf.net.rolling(100).mean().tail()